# Introducción al aprendizaje automático: Práctica 2

En esta práctica se desarrollaran distintas variaciones del algorimo de regresión linear múltiple y se comparara con la implementación de la libreria `scikit-learn`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Para la implementación de la función de regresión lineal múltiple he decidido hacerlo como un todo en uno, donde podremos variar la regularización y las funciones de error.

In [ ]:
import numpy as np

def linear_regression(X: np.ndarray, y: np.ndarray, 
                      error_function: callable, regularization_derivative: callable, 
                      epsilon: float, lambda_reg: float, learning_rate: float, 
                      epochs: int, batch_size: float = 1.0) -> np.ndarray:
    """
    Realiza la regresión lineal múltiple con descenso de gradiente y regularización.

    Args:
        X (np.ndarray): Matriz de características (normalizada y con bias).
        y (np.ndarray): Vector de etiquetas.
        error_function (callable): Función para calcular el error.
        regularization_derivative (callable): Derivada de la función de regularización.
        epsilon (float): Umbral de convergencia.
        lambda_reg (float): Peso de la regularización.
        learning_rate (float): Tasa de aprendizaje.
        epochs (int): Número de épocas.
        batch_size (float): Porcentaje del dataset usado en cada batch (0-1).

    Returns:
        np.ndarray: Vector de parámetros ajustados (model).
    """

    assert 0 < batch_size <= 1, "batch_size debe estar en el rango (0, 1]"
    assert X.shape[0] == y.shape[0], "X e y deben tener la misma cantidad de muestras"
    

    dataset_size, features_size = X.shape
    batch = max(1, int(batch_size * dataset_size))  # Asegura al menos 1 muestra por batch
    model = np.random.rand(features_size)  

    prev_loss = float('inf')

    for _ in range(epochs):
        indices = np.random.permutation(dataset_size)  # Barajar datos en cada época
        X_shuffled = X[indices]
        y_shuffled = y[indices]

        for i in range(0, dataset_size, batch):
            X_batch = X_shuffled[i:i + batch]
            y_batch = y_shuffled[i:i + batch]

            regression_error = X_batch @ model - y_batch # y_pred - y
            gradient = X_batch.T @ regression_error + lambda_reg * regularization_derivative(model) # w_i = w_i * error + reg
            model -= learning_rate * gradient  # Actualización de pesos
        
        loss = error_function(X, y, model)
        if abs(prev_loss - loss) < epsilon:
            break
        prev_loss = loss

    return model

Ahora vamos a definir una función de predicción, que nos ayudará como abtracción en nuestra tarea.

In [ ]:
def predict(X: np.ndarray, model: np.ndarray) -> np.ndarray:
    """
    Realiza la predicción de un conjunto de datos.

    Args:
        X (np.ndarray): Matriz de características.
        model (np.ndarray): Vector de parámetros ajustados.

    Returns:
        np.ndarray: Vector de etiquetas predichas.
    """
    return X @ model

Para poder realizar una correcta medición del progreso de nuestro modelo de regresión vamos a crear las dos funciones de error que hemos dado en clase.

- Error medio cuadrático (_MSE_): $\frac{1}{n} \sum^n_{i=1}(y_i - \hat y_i)^2$

- Error medio absoluto (_MAE_): $\frac{1}{n} \sum^n_{i=1}\mid y_i - \hat y_i \mid$

In [ ]:
def mse(X: np.ndarray, y: np.ndarray, model: np.ndarray) -> float:
    return np.mean((X @ model - y) ** 2)

def mae(X: np.ndarray, y: np.ndarray, model: np.ndarray) -> float:
    return np.mean(np.abs(X @ model - y))

Para regularizar usaremos las distintas normas dadas en clase:

- Norma $L_1$: ${\sum^n_{i=1} \mid v_i \mid}$

- Norma $L_2$: $\sqrt{\sum^n_{i=1} v_i^2}$

Para uilizar estas regulariazaciones en el gradiente descendente, usaremos las derivadas de estas normas.

$$\frac{\partial}{\partial L_1} = \frac{\partial}{\partial v} {\sum^n_{i=1} \mid v_i \mid}= \sum^n_{i=1} signo(v_i)$$
$$\frac{\partial}{\partial L_2} = \frac{\partial}{\partial v} {\sum^n_{i=1} v_i^2} = \sum^n_{i=1} 2 \cdot v_i$$


In [ ]:
def l1(model: np.ndarray) -> float: # Lasso
    return np.sum(np.sign(model))

def l2(model: np.ndarray) -> float: # Rigde
    return np.sum(2 * model)

Ahora vamos a hacer una prueba con los datos que nos vienen en el enunciado.

In [ ]:
X = np.array([
    [1, 2],
    [1, 3],
    [2, 3],
    [2, 4],
    [3, 2],
    [3, 5],
    [4, 1]
])

y = np.array([1.03, -1.44, 4.53, 2.24, 13.27, 5.62, 21.53])

In [ ]:
error_function = np.random.choice([mse, mae])

lasso = [error_function, l1]
ridge = [error_function, l2]

choice = np.random.choice([0, 1])

print(f'Error tipo: {"Lasso" if choice == 0 else "Ridge"}')

my_model = linear_regression(X, y, *[lasso, ridge][choice], 1e-6, 0.1, 0.01, 1000, 0.5)
print(f'Error de nuestro modelo ({error_function.__name__.upper()}): {error_function(X, y, my_model)}')

In [ ]:
predictions = predict(X, my_model)

Vamos a representar las predicciones de nuestro modelo.

In [ ]:
def plot_predictions(y, predictions):
    plt.figure(figsize=(10, 6))
    plt.plot(y, label='Valores reales', marker='o')
    plt.plot(predictions, label='Predicciones', marker='x')
    plt.xlabel('Índice')
    plt.ylabel('Valor')
    plt.title('Comparación de valores reales y predicciones')
    plt.legend()
    plt.grid(True)
    plt.show()

plot_predictions(y, predictions)

Ahora vamos a probar con la función implementada de la libreria `sklearn` y la compararemos con nuestro algoritmo sobre el conjunto de datos de diabetes.

In [ ]:
import sklearn.linear_model as lm
import sklearn.datasets as ds

In [ ]:
X, y = ds.load_diabetes(return_X_y=True)

# Creación del modelo y predicción de los datos (Mí algoritmo)
error_function = np.random.choice([mse, mae])

lasso = [error_function, l1]
ridge = [error_function, l2]

choice = np.random.choice([0, 1])

my_diabetes_model = linear_regression(X, y, *[lasso, ridge][choice], 1e-7, 0.2, 0.1, 6000, 0.1)
my_predictions = predict(X, my_diabetes_model)

# Creación del modelo y predicción de los datos (Modelo de sklearn)

sklearn_model = lm.LinearRegression()
sklearn_model.fit(X, y)
sklearn_predictions = sklearn_model.predict(X)

print('Mis predicciones:')
print('Mi Error: ', mse(X, y, my_diabetes_model))
plot_predictions(y, my_predictions)

print('Predicciones de sklearn:')
print('Error de sklearn: ', mse(X, y, sklearn_model.coef_))
plot_predictions(y, sklearn_predictions)

Ahora haremos uso de los mismos algoritmos de sklearn pero con terminos de regulariazción $L_1$, $L_2$ y $ElasticNet$

In [ ]:
lasso_model = lm.Lasso(alpha=0.1)
lasso_model.fit(X, y)
lasso_predictions = lasso_model.predict(X)

print('Predicciones de Lasso:')
print(f'Error de Lasso: {mse(X, y, lasso_model.coef_)}')
plot_predictions(y, lasso_predictions)

In [ ]:
ridge_model = lm.Ridge(alpha=0.1)
ridge_model.fit(X, y)
ridge_predictions = ridge_model.predict(X)

print('Predicciones de Ridge:')
print('Error de Ridge: ', mse(X, y, ridge_model.coef_))
plot_predictions(y, ridge_predictions)

In [ ]:
elastic_net_model = lm.ElasticNet(alpha=0.005, l1_ratio=0.85)
elastic_net_model.fit(X, y)
elastic_net_predictions = elastic_net_model.predict(X)

print('Predicciones de Elastic Net:')
print('Error de Elastic Net: ', mse(X, y, elastic_net_model.coef_))
plot_predictions(y, elastic_net_predictions)